<a href="https://colab.research.google.com/github/Nikhitaa2329/genAI1/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install PyTorch if not already installed (in Colab, usually pre-installed)
# !pip install torch

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate dummy sequence data: more 1s → class 1, more 0s → class 0
def generate_data(num_samples=1000, seq_len=10):
    X = []
    y = []
    for _ in range(num_samples):
        seq = np.random.randint(0, 2, size=seq_len)
        label = int(seq.sum() > seq_len // 2)
        X.append(seq)
        y.append(label)
    return np.array(X), np.array(y)

# Generate data
X, y = generate_data()

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # Shape: (batch, seq_len, 1)
y_tensor = torch.tensor(y, dtype=torch.long)

# Create dataset and dataloader
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=16, num_layers=1, num_classes=2):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # Get output from last time step
        out = self.fc(out)
        return out

# Initialize model, loss function, and optimizer
model = SimpleRNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10):
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

# Test the model on a custom sequence
test_seq = np.array([[1, 0, 1, 1, 0, 1, 1, 1, 0, 0]])  # Should be class 1
test_tensor = torch.tensor(test_seq, dtype=torch.float32).unsqueeze(-1).to(device)

with torch.no_grad():
    pred = model(test_tensor)
    predicted_class = torch.argmax(pred, dim=1).item()

print(f"Predicted class: {predicted_class}")


Epoch 1 | Loss: 0.3191
Epoch 2 | Loss: 0.0256
Epoch 3 | Loss: 0.0352
Epoch 4 | Loss: 0.0813
Epoch 5 | Loss: 0.1199
Epoch 6 | Loss: 0.0027
Epoch 7 | Loss: 0.0358
Epoch 8 | Loss: 0.1031
Epoch 9 | Loss: 0.0190
Epoch 10 | Loss: 0.0153
Predicted class: 1
